- There are three raw data sources that we have to transform and merge in one dataframe to perform analysis 
- Since the data is based on three states-- CA, TX and WI-- we can create dataset for each of the states to lower the memory allocation and total size of the dataset. Without splitting the dataset, the total space is 4+ and contains more than hundreds of millions of rows 
    - splitting the dataset into the three states will provide us better flexiability in terms of memory allocation and better performance
- We can also create a sample dataset for each products in different states by measuring the confidence interval and the errors of sample values not being the true values 



In [1]:
import pandas as pd 
import numpy as np 
import gc

In [5]:
def transform_df(): 
    calender_dtype={"event_name_1": "category", "event_name_2": "category", "event_type_1": "category", 
         "event_type_2": "category", "weekday": "category", 'wm_yr_wk': 'int16', "wday": "int16",
        "month": "int16", "year": "int16", "snap_CA": "uint8", 'snap_TX': 'uint8', 'snap_WI': 'uint8' }
    calender = pd.read_csv('calendar.csv', dtype= calender_dtype)
    calender.date = pd.to_datetime(calender.date)
         
    price_dtype = {"store_id": "category", "item_id": "category", "wm_yr_wk": "int16","sell_price":"float32" }
    #prices of products 
    product_prices = pd.read_csv('sell_prices.csv', dtype= price_dtype)  
  
    days = 1913
    colstype = ['id', "store_id", 'state_id', 'item_id', 'cat_id', 'dept_id']
    d_numtype = [f"d_{day}" for day in range(1,days+1)]
    
    dtype = {num: 'int8' for num in d_numtype}
    dtype.update({col: "category" for col in colstype}) 
    
    daily_temp = pd.read_csv('sales_train_validation.csv', usecols = d_numtype + colstype, dtype = dtype)
    
    daily_sales = pd.melt(daily_temp,id_vars = ['id', 'store_id', 'item_id','state_id','cat_id', 'dept_id'], 
                      var_name = "d", value_name = 'Sales')
    
    gc.collect()
    
    daily_sales = daily_sales.merge(calender, on = 'd', copy = 'false')
    dataset = daily_sales.merge(product_prices, on = ['store_id', 'item_id', 'wm_yr_wk'], copy = False)
    
    dataset = dataset.drop(columns = ['id', 'd', 'cat_id', 'month', 'year', 'weekday', 'wday']) #since they are almost duplicates, we can drop the columns. 
    gc.collect() #free memory 

    return dataset


In [6]:
dataset = transform_df()

In [7]:
CA_df = dataset[dataset['state_id'] == 'CA']
TX_df = dataset[dataset['state_id'] == 'TX']
WI_df = dataset[dataset['state_id'] == 'WI']

gc.collect()


20

In [10]:
#CA_df.to_pickle("CA_df.pkl")
#WI_df.to_pickle('WI_df.pkl')
#TX_df.to_pickle('TX_df.pkl')